In [1]:
# Import the necessary libraries
import numpy as np
import scipy
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras import utils
import matplotlib.pyplot as plt
import glob

from model import net
%matplotlib inline
#from sklearn.cross_validation import StratifiedKFold

In [2]:
BATCHSZ = 32
EPOCHS = 150

In [3]:
data_dir= "E:\Eric_HSI\hyper_data_preprocess\Indian_pines_w_size_9_num_200_for_2D"
data_root = glob.glob(data_dir + '/*')
print(data_root)

['E:\\Eric_HSI\\hyper_data_preprocess\\Indian_pines_w_size_9_num_200_for_2D\\data.npy', 'E:\\Eric_HSI\\hyper_data_preprocess\\Indian_pines_w_size_9_num_200_for_2D\\data_label.npy', 'E:\\Eric_HSI\\hyper_data_preprocess\\Indian_pines_w_size_9_num_200_for_2D\\test.npy', 'E:\\Eric_HSI\\hyper_data_preprocess\\Indian_pines_w_size_9_num_200_for_2D\\test_label.npy', 'E:\\Eric_HSI\\hyper_data_preprocess\\Indian_pines_w_size_9_num_200_for_2D\\train.npy', 'E:\\Eric_HSI\\hyper_data_preprocess\\Indian_pines_w_size_9_num_200_for_2D\\train_label.npy']


In [4]:
for name in glob.glob(data_dir + '/*'):
    print(name)

E:\Eric_HSI\hyper_data_preprocess\Indian_pines_w_size_9_num_200_for_2D\data.npy
E:\Eric_HSI\hyper_data_preprocess\Indian_pines_w_size_9_num_200_for_2D\data_label.npy
E:\Eric_HSI\hyper_data_preprocess\Indian_pines_w_size_9_num_200_for_2D\test.npy
E:\Eric_HSI\hyper_data_preprocess\Indian_pines_w_size_9_num_200_for_2D\test_label.npy
E:\Eric_HSI\hyper_data_preprocess\Indian_pines_w_size_9_num_200_for_2D\train.npy
E:\Eric_HSI\hyper_data_preprocess\Indian_pines_w_size_9_num_200_for_2D\train_label.npy


In [5]:
train = np.load(data_root[4])
train_label = np.load(data_root[5])
test = np.load(data_root[2])
test_label = np.load(data_root[3])
train.shape, train_label.shape, test.shape, test_label.shape

((2460, 9, 9, 200), (2460,), (7789, 9, 9, 200), (7789,))

In [6]:
# Reshape data into (numberofsumples, channels, height, width)

# convert class labels to on-hot encoding
train_label = utils.to_categorical(train_label)
test_label = utils.to_categorical(test_label)
train.shape, train_label.shape, test.shape, test_label.shape

((2460, 9, 9, 200), (2460, 16), (7789, 9, 9, 200), (7789, 16))

In [7]:
train.dtype, test.dtype, train_label.dtype, test_label.dtype

(dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'))

In [8]:
db_train = tf.data.Dataset.from_tensor_slices((train, train_label))
db_test = tf.data.Dataset.from_tensor_slices((test, test_label))

db_train = db_train.shuffle(train.shape[0]).repeat().batch(batch_size=BATCHSZ)
db_test = db_test.batch(batch_size=BATCHSZ)

In [9]:
db_train, db_test

(<BatchDataset shapes: ((None, 9, 9, 200), (None, 16)), types: (tf.float32, tf.float32)>,
 <BatchDataset shapes: ((None, 9, 9, 200), (None, 16)), types: (tf.float32, tf.float32)>)

In [10]:
model = net(im_width=9, im_height=9, im_channel=200 ,num_classes=16, include_top=True)
model.build(input_shape=(None, 9, 9, 204))
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 9, 9, 200)]       0         
_________________________________________________________________
net_block (NetBlock)         (None, 31, 31, 256)       1324876   
_________________________________________________________________
global_average_pooling2d (Gl (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               65792     
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)              

In [11]:
if not os.path.exists('model_indian'):
    os.mkdir('model_indian')

In [12]:
# Define optimization and train method
# Reduce learning rate when a metric has stopped improving.
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.6, patience=10, min_lr=0.000001, verbose=1)
callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath='./model_indian/indian.h5',
                                                save_best_only=True,
                                                # save_weights_only=True,
                                                monitor='val_loss')]
# checkpointer = ModelCheckpoint(filepath=".\checkP\.checkpoint.ckpt", verbose=1, save_best_only=False)
adam = tf.keras.optimizers.Adam(learning_rate=0.001)
# sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(
              optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])

In [13]:
BATCHSZ = 32
EPOCHS = 150

In [ ]:
# Start to train model 
history = model.fit(db_train,
                    batch_size=BATCHSZ, 
                    steps_per_epoch=train.shape[0]//BATCHSZ,
                    epochs=EPOCHS, 
                    validation_data=db_test, 
                    validation_steps=test.shape[0]//BATCHSZ,
                    callbacks=[reduce_lr, callbacks])